This notebook demonstrates the "Grouped" classification paradigm for Common Spatial Patterns based classification of EEG Motor Imagery Trials. It can identify four classes of Motor Imagery - 
* Left Hand
* Right Hand
* Tongue
* Foot

Three binary classifiers are used to classify EEG trials - 
* level 1 classifier
    * "Left Hand or Right Hand" verus "Tongue or Foot" - Decides whether an instance is either Left Hand or Right Hand movement OR Tongue or Foot Movement

* level 2 classifiers
    * Left Hand versus Right Hand movement - Decides whether an instance is Left Hand movement or Right Hand movement
    * Tongue versus Foot movement - Decides whether an instances is Tongue movement or Foot movement

The decision of the level 1 classifier indicates which level 2 classifier to invoke. If the level 1 classifier classifies a trial as Left Hand or Right Hand movement, the level 2 classifier corresponding to Left Hand versus Right Hand movement is invoked to make the final decision.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [ ]:
import numpy as np
import math
import time
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_recall_fscore_support

bands = 10

Utility functions to read data into memory, reshape data, and reformat labels to grouped classification

In [ ]:
def get_data(subject):
    """
    Loads the augmented, filtered data into memory

    Parameters: 
    subject(int): The numeric identifier of each subject in the dataset

    Returns:
    numpy.ndarray: training data
    numpy.ndarray: training labels
    numpy.ndarray: test data
    numpy.ndarray: test labels
    """

    X_train = np.load('/content/drive/My Drive/EEG-MI/data/filtered/X0{}T.npy'.format(subject))
    y_train = np.load('/content/drive/My Drive/EEG-MI/data/filtered/y0{}T.npy'.format(subject))
    X_test = np.load('/content/drive/My Drive/EEG-MI/data/filtered/X0{}E.npy'.format(subject))
    y_test = np.load('/content/drive/My Drive/EEG-MI/data/filtered/y0{}E.npy'.format(subject))
    return X_train, y_train, X_test, y_test

def binarize_labels(y, class_1, class_2):
    """
    Converts an array of labels into binary representation e.g. the labels 770,771 will translate to 0,1.
    """
    y_bin = []
    for i in y:
        if i == class_1:
            y_bin.append(0)
        else:
            y_bin.append(1)
    return np.array(y_bin)

def reshape_trial_major(data):
    """
    Reshapes data to conform to the following axis representation - (trials, frequency bands, channels, samples) 
    e.g. a set with 20 trials, 10 frequency bands, 22 channels and 750 data points gets shaped into a matrix with dimensions (20, 10, 22, 750)

    Parameters: 
    data(numpy.ndarray): EEG signals with the following dimensions - (frequency bands, trials, channels, samples)

    Returns:
    numpy.ndarray: EEG signals with the following dimensions - (trials, frequency bands, channels, samples)
    """
    X = []
    for trial in range(data.shape[1]):
        band_data = []
        for band in range(10):
            band_data.append(data[band][trial])
        X.append(band_data)
    return np.array(X)

def reshape_band_major(data):
    """
    Reshapes data to conform to the following axis representation - (frequency bands, trials, channels, samples) 
    e.g. a set with 20 trials, 10 frequency bands, 22 channels and 750 data points gets shaped into a matrix with dimensions (10, 20, 22, 750)

    Parameters: 
    data(numpy.ndarray): EEG signals with the following dimensions - (trials, frequency bands, channels, samples)

    Returns:
    numpy.ndarray: EEG signals with the following dimensions - (frequency bands, trials, channels, samples)
    """
    X = []
    for band in range(10):
        trial_data = []
            for trial in range(data.shape[0]):
                trial_data.append(data[trial][band])
        X.append(trial_data)
    return np.array(X)

def get_two_class_data(X_train, y_train, X_test, y_test, class_1, class_2):
    """
    Collects trials corresponding to two specified classes

    Parameters:
    X_train(numpy.ndarray): training data in band major format
    y_train(numpy.ndarray): labels corresponding to training data trials
    X_test(numpy.ndarray): testing data in band major format
    y_test(numpy.ndarray): labels corresponding to testing data trials 

    Returns:
    numpy.ndarray: training instances belonging to the two specified classes in band major format
    numpy.ndarray: binary training labels corresponding to the filtered training instances
    """

    X_train = reshape_trial_major(X_train)
    train = np.array([X_train[i] for i in range(len(y_train)) if y_train[i] in [class_1, class_2]])
    train = reshape_band_major(train)
    train_labels = [i for i in y_train if i in [class_1, class_2]]
    
    X_test = reshape_trial_major(X_test)
    test = np.array([X_test[i] for i in range(len(y_test)) if y_test[i] in [class_1, class_2]])
    test = reshape_band_major(test)
    test_labels = [i for i in y_test if i in [class_1, class_2]]

    return(train, binarize_labels(train_labels, class_1, class_2), test, binarize_labels(test_labels, class_1, class_2))

def get_group_labels(y_train, y_test):
    """
    Groups labels into two buckets - those corresponding to class 769 and 770 (assigned a class of 0), and thos belonging to class 771 and 772 (assigned
    a class of 1)

    Parameters:
    y_train(numpy.ndarray): the labels corresponding to the training instances
    y_test(numpy.ndarray): the labels corresponding to the test instances

    Returns:
    list: the grouped binary labels corresponding to the training instances
    list: the grouped binary labels corresponding to the test instances
    """
    train_labels = []
    for i in y_train:
        if i in [769, 770]:
            train_labels.append(0)
        else:
            train_labels.append(1)
    test_labels = []
    for i in y_test:
        if i in [769, 770]:
            test_labels.append(0)
        else:
            test_labels.append(1)
    return(train_labels, test_labels)

Functions to construct the Common Spatial Patterns projection matrices and compute CSP features on input data

In [5]:
def get_CSP_mat(X,y,class1,class2):
    """
    Generates the CSP projection matrix for the input training instances

    Parameters:
    X(numpy.ndarray): Training instances corresponding to a particular frequency band
    y(numpy.ndarray): Training labels
    class1: the value of the label corresponding to the first class
    class2: the value of the label corresponding to the second class

    Returns:
    numpy.ndarray: The CSP Projection Matrix
    numpy.ndarray: The sorted Eigenvalues
    """

    R1=np.zeros((22,22)) #To store the sum of the covariance matrices of class 1
    R2=np.zeros((22,22)) #To store the sum of the covariance matrices of class 1
    
    c1=list(y).count(class1) 
    c2=list(y).count(class2)
 
    #Compute the sum of covariance matrices for each trial for each class
    for i in range(len(y)):
        temp=np.dot(X[i],np.transpose(X[i]))
        temp=np.divide(temp,np.trace(temp))
        if(y[i])==class1:
            R1=np.add(R1,temp)
        else:
            R2=np.add(R2,temp)

    #Compute average covariance matrices for both classes 
    R1=np.divide(R1,c1)
    R2=np.divide(R2,c2)
    R=np.add(R1,R2) #Composite Spatial Covariance
    
    e_val,e_vec=np.linalg.eig(R)
    #Sort in Descending Order of eigenvalues
    e_val=[(e_val[i],i) for i in range(len(e_val))]
    e_val=list(e_val)
    e_val.sort(key= lambda x:x[0])
    e_val.reverse()
    args=[e_val[i][1] for i in range(22)]
    e_val=[e_val[i][0] for i in range(22)]
    e_vec=e_vec[:,args]
    e_val=np.diag(np.power(e_val,-0.5))

    P=np.dot(e_val,np.transpose(e_vec))#Whitening Transformation Matrix

    S1=np.dot(P,np.dot(R1,np.transpose(P)))
    S2=np.dot(P,np.dot(R2,np.transpose(P)))

    #Simultaneous diagonalisation of S1 and S2
    e_val1,e_vec=np.linalg.eig(S1)
    e_val2,e_vec=np.linalg.eig(S2)
    
    a=np.argsort(e_val1) #Sort eigenvalues 
    W=np.dot(np.transpose(e_vec),P) #Final Projection Matrix

    return(W,a)

def get_CSP_band_features(data,W,a):
    """
    Computes the log variance of the CSP transformed signals

    Parameters: 
    data(numpy.ndarray): The input trials corresponding to a single frequency band
    W(numpy.ndarray): The CSP projection matrix for this frequency band
    a(numpy.ndarray): The sorted Eigenvalues

    Returns:
    numpy.ndarray: The log variance of the channels of CSP transformed signals corresponding to the two minimum and maximum eigenvalues
    """

    reconstructed_eeg=np.dot(W,data)
    sources=[reconstructed_eeg[k] for k in [a[0],a[1],a[20],a[21]]]
    sources_var=[]
    s=0
    for j in sources:
        k=np.var(j)
        s+=k
        sources_var.append(k)
    return(np.log(np.divide(sources_var,s)))

def get_test_features(data, W, a):
    """
    Generates the CSP features for all frequency bands of the input signals

    Parameters:
    data(numpy.ndarray): The input trials corresponding to all frequency bands
    W(numpy.ndarray): The CSP projection matrix for each frequency band
    a(numpy.ndarray): The sorted Eigenvalues for each frequency band
    
    Returns:
    numpy.ndarray: The log variance of the channels of CSP transformed signals corresponding to the two minimum and maximum eigenvalues for all frequency bands
    """
  
    features = []
    for i in range(10):
        features.extend(get_CSP_band_features(data[i], W[i], a[i]))
    return np.array(features)

def get_features(X_train, X_test, y_train):
    """
    Computes the train and test CSP features for the input train and test signals

    Parameters: 
    X_train(numpy.ndarray): The input training set of trials
    X_test(numpy.ndarray): The input test set of trials
    y_train(numpy.ndarray): The labels corresponding to the training set 

    Returns:
    numpy.ndarray: the log variance features of the CSP transformed training signals
    numpy.ndarray: the log variance features of the CSP transformed test signals
    numpy.ndarray: the CSP Projection Matrices for each frequency band
    numpy.ndarray: the eigenvalues for each frequency band
    """
    W = []
    a = []

    for band in range(bands):
        W_band, a_band = get_CSP_mat(X_train[band],y_train, 0, 1)
        W.append(W_band)
        a.append(a_band)

    features_train = []
    for i in range(X_train.shape[1]):
        features = []
        for band in range(bands):
            features.extend(get_CSP_band_features(X_train[band][i], W[band], a[band]))
        features_train.append(features)

    features_train = np.array(features_train)

    features_test = []
    for i in range(X_test.shape[1]):
        features = []
        for band in range(bands):
            features.extend(get_CSP_band_features(X_test[band][i], W[band], a[band]))
        features_test.append(features)

    features_test = np.array(features_test)

    return(features_train, features_test, W, a)

Functions to train the grouped classifier

In [3]:
def build_one_vs_one_clf(X_train, y_train, X_test, y_test, class_1, class_2, lr, depth, est, cols, level_1=False):
    """
    Builds one versus one classifiers for the provided class labels. 

    Parameters:
    X_train(numpy.ndarray): the training instances
    y_train(numpy.ndarray): the training labels
    X_test(numpy.ndarray): the test instancs
    y_test(numpy.ndarray): the test labels
    class1(int): the class label of the first class
    class2(int): the class label of the second class
    lr(float): learning rate of XGBoost classifier
    depth(int): the max depth of a tree for the XGBoost classifier
    est(int): number of estimators for the XGBoost classifier
    cols(float): the column subsampling ratio of the XGBoost classfier
    level_1(boolean): indicator of which classification level the classifier belongs to (true if level1, false otherwise)

    Returns:
    xgboost.XGBXlassifier: the trained classifier
    numpy.ndarray: the CSP projection matrices 
    numpy.ndarray: the sorted eigenvalues
    """

    if not(level_1):
      f_train, l_train, f_test, l_test = get_two_class_data(X_train, y_train, X_test, y_test, class_1, class_2)
    else:
      f_train = X_train
      f_test = X_test
      l_train = y_train
      l_test = y_test
    f_train, f_test, W, a = get_features(f_train, f_test, l_train)
    clf = xgboost.XGBClassifier(max_depth=depth, learning_rate=lr, n_estimators=est, objective='binary:logistic', subsample=0.8, colsample_bytree=cols, silent=True)
    clf.fit(f_train, l_train)
    return clf, W, a

def build_classifiers(subject, lr, depth, est, cols):
    """
    constructs the hierarchy of one versus one classifiers. The level 1 classifier distinguishes between two classes - "Left or Right hand" versus "Tongue or Foot". The decision of this classifier indicates which level 2 classifier to invoke.
    If the level 1 classifier predicts "left or right hand" then the left hand verus right hand level 2 classifier is invoked. Else, the tongue versus foot level 2 classifier is invoked.

    Parameters: 
    subject(int): the numeric identifier of the subject
    lr(float): learning rate of XGBoost classifier
    depth(int): the max depth of a tree for the XGBoost classifier
    est(int): number of estimators for the XGBoost classifier
    cols(float): the column subsampling ratio of the XGBoost classfier

    Returns:
    dict: the trained level1 and leve2 classifiers
    numpy.ndarray: the test instances
    numpy.ndarray: the test labels
    """

    X_train, y_train, X_test, y_test = get_data(subject)
    models = {}
    #build level one clf
    y_1_train, y_1_test = get_group_labels(y_train, y_test)
    clf, W, a = build_one_vs_one_clf(X_train, y_1_train, X_test, y_1_test, 0, 1, lr, depth, est, cols, True)
    models['LRvsTF'] = {'clf': clf, 'W':W, 'a':a}

    #build LR clf
    clf, W, a = build_one_vs_one_clf(X_train, y_train, X_test, y_test, 769, 770, lr, depth, est, cols)
    models['LR'] = {'clf': clf, 'W':W, 'a':a}

    #build TF clf
    clf, W, a = build_one_vs_one_clf(X_train, y_train, X_test, y_test, 771, 772, lr, depth, est, cols)
    models['TF'] = {'clf': clf, 'W':W, 'a':a}

    return models, X_test, y_test

Functions to make predictions on test data and evaluate model performance

In [4]:
def score(labels_pred, labels_true):
    """
    Returns the accuracy of the model 

    Parameters:
    labels_pred(numpy.ndarray): The predicted class labels
    labels_true(numpy.ndarray): The ground truth labels

    Returns:
    float: the prediction accuracy
    """
    scores = 0
    for i in range(len(labels_pred)):
        if labels_pred[i] == labels_true[i]:
            scores+=1
    return(scores/len(labels_pred))

def predict(models, X_test, y_test):
    """
    Uses the built models to predict class labels of test instances. The "left or right hand" versus "tongue or foot" classifier is first invoked. The decision of this classifier indicates which level 2 classifier to invoke.
    If the level 1 classifier predicts "left or right hand" then the left hand verus right hand level 2 classifier is invoked. Else, the tongue versus foot level 2 classifier is invoked.

    Parameters:
    models(dict): dictionary of all models
    X_test(numpy.ndarray): the test instances
    y_test(numpy.ndarrat): the labels corresponding to the test set

    Returns:
    float: the prediction accuracy of the model
    float: the cohen kappa score of the model
    np.ndarray: the average precision, recall and f1 scores of the model
    """

    X_test = reshape_trial_major(X_test)
    pred = []
    start = time.time()
    for i in range(len(y_test)):
        clf_LRvsTF = models['LRvsTF']
        sample_LRvsTF = get_test_features(X_test[i], clf_LRvsTF['W'], clf_LRvsTF['a'])
        pred_LRvsTF = clf_LRvsTF['clf'].predict([sample_LRvsTF])[0]

        if pred_LRvsTF == 0:
            clf_LR = models['LR']
            sample_LR = get_test_features(X_test[i], clf_LR['W'], clf_LR['a'])
            pred_LR = clf_LR['clf'].predict([sample_LR])[0]
            pred_LR = 769 if pred_LR == 0 else 770
            pred.append(pred_LR)

        else:
            clf_TF = models['TF']
            sample_TF = get_test_features(X_test[i], clf_TF['W'], clf_TF['a'])
            pred_TF = clf_TF['clf'].predict([sample_TF])[0]
            pred_TF = 771 if pred_TF == 0 else 772
            pred.append(pred_TF)
    end = time.time() - start
    return score(pred, y_test), cohen_kappa_score(pred, y_test), end/len(y_test), np.mean(precision_recall_fscore_support(y_test, pred)[0]), np.mean(precision_recall_fscore_support(y_test, pred)[1]), np.mean(precision_recall_fscore_support(y_test, pred)[2])

Builds the classifiers and evaluates them on each subjects data. The parameter values used for the XGBoost classifiers -


*   Max depth - 8
*   Number of Estimator - 500
*   Column Subsampling Ratio - 0.5


In [ ]:
kappas = []
accs = []
p = []
r = []
f = []
for subject in range(1,10):
  models, X_test, y_test = build_classifiers(subject, 0.01, 8, 500, 0.5)
  acc, kappa, test_time, precision, recall, f1 = predict_one_vs_one(models, X_test, y_test)
  accs.append(acc)
  kappas.append(kappa)
  p.append(precision)
  r.append(recall)
  f.append(f1)
  print(subject, kappa, acc, precision, recall, f1)
  
print("kappa: ", np.mean(kappas), "accs: ",np.mean(accs), "precision: ", np.mean(p), "recall: ", np.mean(recall), "f1: ", np.mean(f))

1 0.7685185185185185 0.8263888888888888 0.8442493169559329 0.8263888888888888 0.8250613991600708
2 0.43981481481481477 0.5798611111111112 0.5803960911822486 0.579861111111111 0.5674835902822569
3 0.7314814814814814 0.7986111111111112 0.8048105582941649 0.7986111111111112 0.7950489521168838
4 0.4722222222222222 0.6041666666666666 0.670702902719483 0.6041666666666666 0.5801048357628964
5 0.3194444444444444 0.4895833333333333 0.5186104574413132 0.4895833333333333 0.39948227414897697
6 0.20833333333333337 0.40625 0.46757362736701874 0.40625 0.3810042749059739
7 0.7962962962962963 0.8472222222222222 0.8552816352004993 0.8472222222222222 0.847032967032967
8 0.6527777777777778 0.7395833333333334 0.7651046147707578 0.7395833333333333 0.7384933041938542
9 0.37962962962962965 0.5347222222222222 0.6601779422128259 0.5347222222222222 0.5023845635321874
kappa:  0.5298353909465021 accs:  0.6473765432098765 precision:  0.6852119051271381 recall:  0.5347222222222222 f1:  0.6262329067928962
